importing all the necessary libraries 

In [ ]:
import cv2 as cv
import os 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import random
import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout , Activation, Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import applications


getting the data from the input folder using list dir and
using cv to convert the image to np array
resizing the image,
normalizing the image
and appending to the list

In [ ]:
X=[];y=[]
for i in os.listdir('../input/screw-dataset/train'):
  gray=cv.resize(cv.imread('../input/screw-dataset/train/{}'.format(i), cv.IMREAD_COLOR),(256,256), interpolation=cv.INTER_CUBIC)
  gray=gray/255.0
  X.append(gray)
  if (i.find('ok') == -1):
      y.append(0)
  else :
      y.append(1)


as it is a skew dataset we need more data so we duplicate the given data 

In [ ]:
a=250;b=50
while (a!=b):
    p=random.randint(0,len(X)-1)
    if y[p]==0:
        b=b+1;
        X.append(X[p])
        y.append(0)


reshaping the X array and coverting the y list to a categorical list

In [ ]:
X=np.array(X)
X=np.reshape(X,[X.shape[0],256,256,3])
y=np.array(y)
y = np_utils.to_categorical(y, 2)
X.shape

(500, 256, 256, 3)

spliting the data into train and validation sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=0)

data augumentation

In [ ]:

train_datagen = ImageDataGenerator(rotation_range=360 )

train_generator = train_datagen.flow(X, y, batch_size=32)


using the resnet50 model as a base model and then adding dense layers to the flattened model

In [ ]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(256,256,3),classes=y_train.shape[1])


model=Sequential()

model.add(base_model_resnet)
model.add(Flatten())

model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dropout(.4))
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
model.add(Dropout(.2))
model.add(Dense(2,activation=('softmax')))

compiling the model using here we specify the optimizer as the adam optimizer and metric as accuracy and loss as categorical crossentropy

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

 train the model using the train data and the validation data is used as the validation data

In [ ]:

history = model.fit_generator(train_generator,
                              steps_per_epoch=len(X_train) // 32,
                              epochs=250,
                              validation_data=(X_val,y_val),
                              validation_steps=len(X_val)//32)

12/12 [==============================] - 12s 677ms/step - loss: 0.4263 - accuracy: 0.8091 - val_loss: 0.3527 - val_accuracy: 0.9300


test data is taken form the Directory and converted as np array and then resized and Normalised and appended to the test list

In [ ]:
test=[];id=[]
for i in os.listdir('../input/screw-dataset/test'):
  id.append(i)
id.sort(key=lambda item: (len(item), item))


In [ ]:
for i in id:
  gray=cv.resize(cv.imread('../input/screw-dataset/test/{}'.format(i), cv.IMREAD_COLOR),(256,256), interpolation=cv.INTER_CUBIC)
  gray=gray/255.0
  test.append(gray)

test list is converted into numpy array and reshaped

In [ ]:
test=np.array(test)
test=np.reshape(test,[test.shape[0],256,256,3])

we use the model to predict the values form the given input test data

In [ ]:
pred=model.predict(test)

In [ ]:
k=[];c=0
for i in range(test.shape[0]):
    if pred[i][0]>pred[i][1]:
        k.append(0)
        c=c+1
        
    else:
        k.append(1)

In [ ]:
df = pd.DataFrame()
df["id"]=id
df["predicted"]=k
df.to_csv("result-submision.csv")